# Complete Petrophysical Workflow

This notebook demonstrates a complete petrophysical analysis workflow using GeoSuite, from data loading to reservoir characterization.

## Overview

This comprehensive workflow shows how to:

1. Load well log data (LAS format)
2. Calculate petrophysical properties (porosity, water saturation)
3. Create diagnostic crossplots (Pickett, Buckles)
4. Classify reservoir quality zones
5. Visualize results with professional plots

This demonstrates best practices for using GeoSuite's integrated petrophysical analysis tools.


In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
from geosuite.petro import (
    calculate_water_saturation,
    calculate_porosity_from_density,
    pickett_plot,
    buckles_plot
)
from geosuite.plotting import create_strip_chart

print("GeoSuite imported successfully!")


## 1. Load Well Log Data

Load well log data using GeoSuite's data loaders. In practice, you would use `load_las()` from `geosuite.io.las_loader`.


In [ ]:
# Load well log data
# In practice: from geosuite.io.las_loader import load_las
# df = load_las('path/to/well.las')

# For demonstration, use demo data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")

# Identify key columns
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
porosity_cols = [col for col in df.columns if 'PHI' in col.upper() or 'NPHI' in col.upper() or 'POR' in col.upper()]
resistivity_cols = [col for col in df.columns if 'RT' in col.upper() or 'RES' in col.upper() or 'ILD' in col.upper()]
density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]

porosity_col = porosity_cols[0] if porosity_cols else None
resistivity_col = resistivity_cols[0] if resistivity_cols else None
density_col = density_cols[0] if density_cols else None

print(f"\nDepth column: {depth_col}")
print(f"Porosity column: {porosity_col}")
print(f"Resistivity column: {resistivity_col}")
print(f"Density column: {density_col}")

df.head()


## 2. Calculate Petrophysical Properties

Use GeoSuite's petrophysical calculation functions to compute porosity and water saturation.


In [ ]:
# Calculate porosity from density (if density log available)
if density_col:
    df['PHI_DENSITY'] = calculate_porosity_from_density(
        df[density_col].values,
        rho_matrix=2.65,  # Sandstone matrix density (g/cc)
        rho_fluid=1.0     # Fresh water fluid density (g/cc)
    )
    print(f"Calculated density porosity")
    print(f"  Range: {df['PHI_DENSITY'].min():.3f} to {df['PHI_DENSITY'].max():.3f}")
    porosity_for_sw = df['PHI_DENSITY'].values
else:
    print("No density log - using available porosity")
    porosity_for_sw = df[porosity_col].values if porosity_col else None

# Calculate water saturation using Archie equation
if resistivity_col and porosity_for_sw is not None:
    df['SW'] = calculate_water_saturation(
        phi=porosity_for_sw,
        rt=df[resistivity_col].values,
        rw=0.05,  # Formation water resistivity (ohm-m)
        m=2.0,    # Cementation exponent
        n=2.0,    # Saturation exponent
        a=1.0     # Tortuosity factor
    )
    
    # Calculate hydrocarbon saturation
    df['SH'] = 1.0 - df['SW']
    
    print(f"\nWater saturation calculated")
    print(f"  Sw range: {df['SW'].min():.3f} to {df['SW'].max():.3f}")
    print(f"  Mean Sw: {df['SW'].mean():.3f}")
    print(f"  Hydrocarbon zones (Sw < 0.5): {(df['SW'] < 0.5).sum()} points")
else:
    print("Cannot calculate water saturation - missing resistivity or porosity")

df.head()


## 3. Create Diagnostic Crossplots

Use GeoSuite's plotting functions to create professional petrophysical crossplots for reservoir quality analysis.


In [ ]:
# Create Pickett plot (porosity vs resistivity)
if porosity_col and resistivity_col:
    phi_col = 'PHI_DENSITY' if density_col else porosity_col
    
    fig1 = pickett_plot(
        df=df,
        porosity_col=phi_col,
        resistivity_col=resistivity_col,
        m=2.0,
        n=2.0,
        a=1.0,
        rw=0.05,
        title='Pickett Plot - Reservoir Quality Analysis',
        show_water_line=True,
        show_sw_lines=True
    )
    plt.show()
else:
    print("Cannot create Pickett plot - missing porosity or resistivity data")


In [ ]:
# Create Buckles plot (porosity vs bulk volume water)
if 'SW' in df.columns and porosity_col:
    phi_col = 'PHI_DENSITY' if density_col else porosity_col
    
    fig2 = buckles_plot(
        df=df,
        porosity_col=phi_col,
        sw_col='SW',
        cutoff=0.04,  # BVW cutoff
        title='Buckles Plot - Pay Zone Identification'
    )
    plt.show()
else:
    print("Cannot create Buckles plot - missing Sw or porosity data")


## 4. Classify Reservoir Quality

Use calculated properties to classify reservoir zones into pay, fair, and non-pay categories.


In [ ]:
# Calculate Bulk Volume Water (BVW)
if 'SW' in df.columns and porosity_col:
    phi_col = 'PHI_DENSITY' if density_col else porosity_col
    df['BVW'] = df[phi_col] * df['SW']
    
    # Classify reservoir quality using BVW cutoff
    bvw_cutoff = 0.04
    df['Reservoir_Quality'] = pd.cut(
        df['BVW'],
        bins=[0, bvw_cutoff, 0.1, 1.0],
        labels=['Pay', 'Fair', 'Non-Pay']
    )
    
    # Additional classification using Sw cutoff
    df['Pay_Zone'] = (df['SW'] < 0.5) & (df['BVW'] < bvw_cutoff)
    
    print("Reservoir Quality Classification:")
    print("=" * 50)
    print(df['Reservoir_Quality'].value_counts())
    print(f"\nPay zones (Sw < 0.5 and BVW < {bvw_cutoff}): {df['Pay_Zone'].sum()} points")
    print(f"  Percentage: {df['Pay_Zone'].sum() / len(df) * 100:.1f}%")
else:
    print("Cannot classify reservoir quality - missing required data")


## 5. Visualize Results

Create professional well log displays showing calculated properties and reservoir classifications.


In [ ]:
# Create strip chart with key logs and calculated properties
log_cols_to_plot = []

# Add original logs
if porosity_col:
    log_cols_to_plot.append(porosity_col)
if resistivity_col:
    log_cols_to_plot.append(resistivity_col)
if density_col:
    log_cols_to_plot.append(density_col)

# Add calculated properties
if 'SW' in df.columns:
    log_cols_to_plot.append('SW')
if 'PHI_DENSITY' in df.columns:
    log_cols_to_plot.append('PHI_DENSITY')

if log_cols_to_plot:
    try:
        fig = create_strip_chart(
            df=df,
            depth_col=depth_col,
            log_cols=log_cols_to_plot[:5],  # Limit to 5 tracks
            title='Complete Petrophysical Analysis - Well Log Display'
        )
        plt.show()
    except Exception as e:
        print(f"Could not create strip chart: {e}")
        print("Falling back to simple plot")
        # Simple fallback
        fig, ax = plt.subplots(figsize=(10, 12))
        if 'SW' in df.columns:
            ax.plot(df['SW'], df[depth_col], 'b-', linewidth=0.5)
            ax.set_xlabel('Water Saturation')
            ax.set_ylabel('Depth')
            ax.invert_yaxis()
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
else:
    print("No logs available for plotting")


## 6. Summary

This complete workflow demonstrated:

- **Data Loading**: Using GeoSuite's data loaders to import well log data
- **Petrophysical Calculations**: Computing porosity and water saturation with `calculate_porosity_from_density()` and `calculate_water_saturation()`
- **Diagnostic Crossplots**: Creating Pickett and Buckles plots with `pickett_plot()` and `buckles_plot()`
- **Reservoir Classification**: Identifying pay zones using BVW and Sw cutoffs
- **Professional Visualization**: Using `create_strip_chart()` for publication-ready displays

### Key GeoSuite Functions Used

- `load_demo_well_logs()` / `load_las()`: Load well log data
- `calculate_porosity_from_density()`: Compute porosity from density log
- `calculate_water_saturation()`: Calculate Sw using Archie equation
- `pickett_plot()`: Create porosity-resistivity crossplot
- `buckles_plot()`: Create BVW-porosity crossplot
- `create_strip_chart()`: Professional well log display

### Next Steps

- Integrate with geomechanical analysis using `geosuite.geomech` functions
- Apply machine learning for facies classification with `geosuite.ml`
- Use change-point detection for automated formation top picking with `geosuite.stratigraphy`
- Export results to your preferred format for further analysis
